In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


def fetch_data(path):
    # fetch dataset 
    X = pd.read_csv(path, delimiter=',')
    X['waterbody'] = X['waterbody'].fillna('No')
    y = X['price']
    X = X.drop(columns="price")

    #X = auto_mpg.data.features 
    #y = auto_mpg.data.targets 
    #X = X.drop(columns=['cylinders'])
    ## Add the targets column to the features
    #X['targets'] = y
    #X = X[X['horsepower'].notna()]
    #y = X['targets']
    #X = X.drop(columns='targets')
    return X, y



# Posebej obravnavamo kategorične atribute: naprimer vejitev waterbody = River in waterbody != River
# Zvezne atribute razdelimo v območja recimo 100 intervalov in za vsako mejo:
# izračunamo povprečje točk na levi strani in povprečje točk na desni strani.
# Seštejemo te dve povprečji in si shranimo to napako za to mejo.
# To naredimo za vsak atribut za vsako mejo in najdemo minimum vseh.
# Potem mamo prvo vozlišče in nato dodajamo. Sami si izberemo kdaj nehamo dodajati


18

In [41]:
class Node:
    def __init__(self, attribute=None, threshold=None, value=None, left=None, right=None):
        self.attribute = attribute  # Name of feature to split on
        self.threshold = threshold  # Threshold value for the feature
        self.value = value  # Prediction value for the node (for leaf nodes)
        self.left = left  # Left subtree
        self.right = right  # Right subtree


class RegressionTree:
    def __init__(self, max_depth=None, min_samples_split=10):
        self.max_depth = max_depth  # Maximum depth of the tree
        self.min_samples_split = min_samples_split  # Minimum samples required to split a node

    def fit(self, X: pd.DataFrame, y):
        self.root = self._fit(X, y, depth=0)
        
    def _fit(self, X: pd.DataFrame, y, depth):
        # Check termination conditions
        if depth == self.max_depth or len(y) < self.min_samples_split or len(set(y)) == 1:
            return Node(value=np.mean(y))

        min_mse = float('inf')
        best_attribute = None
        best_threshold = None

        # Iterate over all features and thresholds to find the best split
        for attr in X.columns.values:
            thresholds = None
            # If current attribute is categoric, iterate through all the distinct values
            if X[attr].dtype == 'object':
                thresholds = list(X[attr].unique())
            else:                    
                thresholds = np.linspace(X[attr].min(), X[attr].max(), 100)
            for threshold in thresholds:
                left_mask = None
                if X[attr].dtype == 'object':
                    left_mask = X[attr] == threshold
                else:
                    left_mask = X[attr] <= threshold
                right_mask = ~left_mask
                if len(y[left_mask]) > 0 and len(y[right_mask]) > 0:
                    mse = self._calculate_mse(y[left_mask]) + self._calculate_mse(y[right_mask])
                    if mse < min_mse:
                        min_mse = mse
                        best_attribute = attr
                        best_threshold = threshold

        if best_attribute is not None:
            left_mask = None
            if X[best_attribute].dtype == 'object':
                left_mask = X[best_attribute] == best_threshold
            else:
                left_mask = X[best_attribute] <= best_threshold
            right_mask = ~left_mask
            left_subtree = self._fit(X[left_mask], y[left_mask], depth + 1)
            right_subtree = self._fit(X[right_mask], y[right_mask], depth + 1)
            return Node(attribute=best_attribute, threshold=best_threshold, left=left_subtree, right=right_subtree)
        else:
            return Node(value=y.mean())

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])

    def _predict(self, x, node: Node):
        if node.value is not None:
            return node.value
        if x[node.attribute].dtype == 'object':
            if x[node.attribute] == node.threshold:
                return self._predict(x, node.left)
            else:
                return self._predict(x, node.right)
        elif x[node.feature_index] <= node.threshold:
            return self._predict(x, node.left)
        else:
            return self._predict(x, node.right)

    def _calculate_mse(self, y):
        return np.mean((y - y.mean())**2)

In [40]:
X, y = fetch_data("House_Price.csv")
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = 

22.528853754940716